In [1]:
import numpy as np
import pandas as pd
import os, glob, pdb

In [2]:
results_dir = 'results'
valid_task_list = ['clock', 'copy', 'trail', 'clock_copy', 'copy_trail', 'copy_trail', 'all']

# Compute the mean and std of each specific setup (e.g., clock + vgg16 + hard labels)


In [3]:
all_dataframe_dict = {}
for curr_task in os.listdir(results_dir):
    if curr_task in valid_task_list:
        for curr_label_type in os.listdir(os.path.join(results_dir, 
                                                       curr_task)): 
            if curr_label_type in ['hard', 'soft']:
                for curr_model in os.listdir(os.path.join(results_dir, 
                                                          curr_task, 
                                                          curr_label_type)):
                    if curr_model in ['vgg16', 'conv-att']:
                        
                        df_many_seeds = pd.DataFrame()
                        for curr_seed in os.listdir(os.path.join(results_dir, 
                                                                 curr_task, 
                                                                 curr_label_type,
                                                                 curr_model)):
                            
                            
                            
                            curr_path = os.path.join(results_dir, 
                                                     curr_task, 
                                                     curr_label_type, 
                                                     curr_model,
                                                     curr_seed)
                            
                            if os.path.isdir(curr_path) and (not curr_seed.startswith(".")):

                                curr_seed_no_date = int(curr_seed.split('_', 1)[0])
                                
                                # Extract result for the current seeds
                                temp_df = pd.read_csv(os.path.join(curr_path, "eval_metrics.csv"))
                                temp_df.insert(0, 'seed', curr_seed_no_date)

                                # Combine it to other seeds
                                df_many_seeds = df_many_seeds.append(temp_df)

                        # Compute the mean
                        df_mean = df_many_seeds.mean(axis=0)
                        df_mean["seed"] = "mean"

                        # Compute the std
                        df_std = df_many_seeds.std(axis=0)
                        df_std["seed"] = "std"
                        
                        # Add the calculated mean and std to the current dataframe
                        df_many_seeds = df_many_seeds.append(df_mean, ignore_index=True) 
                        df_many_seeds = df_many_seeds.append(df_std, ignore_index=True)
                         
                        
                        df_many_seeds.to_csv(os.path.join(results_dir, 
                                                          curr_task, 
                                                          curr_label_type, 
                                                          curr_model, 
                                                          'eval_combined_seeds.csv'),
                                             index=False)
                        
                        all_dataframe_dict[f"{curr_task}_{curr_label_type}_{curr_model}"] = df_many_seeds

In [4]:
with pd.ExcelWriter(os.path.join(results_dir, 'eval_comparison.xlsx'), engine='openpyxl') as writer:
    
    df_mean = pd.DataFrame()
    df_std = pd.DataFrame()
    
    for idx, curr_exp in enumerate(all_dataframe_dict.keys()):
        curr_mean = all_dataframe_dict[curr_exp].iloc[-2, 1:].to_frame().transpose()
        curr_mean.insert(0, 'methods', curr_exp)
        df_mean = df_mean.append(curr_mean)
        
        curr_std = all_dataframe_dict[curr_exp].iloc[-1, 1:].to_frame().transpose()
        curr_std.insert(0, 'methods', curr_exp)
        df_std = df_std.append(curr_std)
        
        all_dataframe_dict[curr_exp].to_excel(writer, sheet_name=curr_exp, index=False)
        
    df_mean.to_excel(writer, sheet_name='mean', index=False)
    df_std.to_excel(writer, sheet_name='std', index=False)